(use/api)=

# Python API

This page outlines how to utilise the cache programatically.
We step throught the three aspects illustrated in the diagram below:
[cacheing](use/api/cache), [staging](use/api/project) and [executing](use/api/execute).

```{figure} images/execution_process.svg
:width: 500 px

Illustration of the execution process.
```

```{note}
The full Jupyter notebook for this page can accessed here; {nb-download}`api.ipynb`.
Try it for yourself!
```

## Initialisation

In [1]:
from pathlib import Path
import nbformat as nbf
from jupyter_cache import get_cache
from jupyter_cache.base import CacheBundleIn
from jupyter_cache.executors import load_executor, list_executors
from jupyter_cache.utils import (
    tabulate_cache_records, 
    tabulate_project_records
)

First we setup a cache and ensure that it is cleared.

```{important}
Clearing a cache wipes its entire content, including any settings (such as cache limit).
```

In [2]:
cache = get_cache(".jupyter_cache")
cache.clear_cache()
cache

JupyterCacheBase('/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/.jupyter_cache')

In [3]:
print(cache.list_cache_records())
print(cache.list_project_records())

[]
[]


(use/api/cache)=

## Cacheing Notebooks

To directly cache a notebook:

In [4]:
record = cache.cache_notebook_file(
    path=Path("example_nbs", "basic.ipynb")
)
record

NbCacheRecord(pk=1)

This will add a physical copy of the notebook to tha cache (stripped of any text cells) and return the record that has been added to the cache database.

```{important}
The returned record is static, as in it will not update if the database is updated.
```

The record stores metadata for the notebook:

In [5]:
record.to_dict()

{'description': '',
 'hashkey': '94c17138f782c75df59e989fffa64e3a',
 'created': datetime.datetime(2022, 1, 12, 15, 15, 27, 255299),
 'accessed': datetime.datetime(2022, 1, 12, 15, 15, 27, 255312),
 'data': {},
 'uri': 'example_nbs/basic.ipynb',
 'pk': 1}

```{important}
The URI that the notebook is read from is stored, but does not have an impact on later comparison of notebooks. They are only compared by their internal content.
```

We can retrive cache records by their Primary Key (pk): 

In [6]:
cache.list_cache_records()

[NbCacheRecord(pk=1)]

In [7]:
cache.get_cache_record(1)

NbCacheRecord(pk=1)

To load the entire notebook that is related to a pk:

In [8]:
nb_bundle = cache.get_cache_bundle(1)
nb_bundle

CacheBundleOut(nb=Notebook(cells=1), record=NbCacheRecord(pk=1), artifacts=NbArtifacts(paths=0))

In [9]:
nb_bundle.nb

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': '1\n'}],
   'source': 'a=1\nprint(a)'}],
 'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
   'file_extension': '.py',
   'mimetype': 'text/x-python',
   'name': 'python',
   'nbconvert_exporter': 'python',
   'pygments_lexer': 'ipython3',
   'version': '3.6.1'},
  'test_name': 'notebook1'},
 'nbformat': 4,
 'nbformat_minor': 2}

Trying to add a notebook to the cache that matches an existing one will result in a error, since the cache ensures that all notebook hashes are unique:

In [10]:
record = cache.cache_notebook_file(
    path=Path("example_nbs", "basic.ipynb")
)

CachingError: Notebook already exists in cache and overwrite=False.

If we load a notebook external to the cache, then we can try to match it to one
stored inside the cache:

In [11]:
notebook = nbf.read(str(Path("example_nbs", "basic.ipynb")), 4)
notebook

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': '# a title\n\nsome text\n'},
  {'cell_type': 'code',
   'execution_count': 1,
   'metadata': {},
   'source': 'a=1\nprint(a)',
   'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': '1\n'}]}],
 'metadata': {'test_name': 'notebook1',
  'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
   'file_extension': '.py',
   'mimetype': 'text/x-python',
   'name': 'python',
   'nbconvert_exporter': 'python',
   'pygments_lexer': 'ipython3',
   'version': '3.6.1'}},
 'nbformat': 4,
 'nbformat_minor': 2}

In [12]:
cache.match_cache_notebook(notebook)

NbCacheRecord(pk=1)

Notebooks are matched by a hash based only on aspects of the notebook that will affect its execution (and hence outputs). So changing text cells will match the cached notebook:

In [13]:
notebook.cells[0].source = "change some text"

In [14]:
cache.match_cache_notebook(notebook)

NbCacheRecord(pk=1)

But changing code cells will result in a different hash, and so will not be matched:

In [15]:
notebook.cells[1].source = "change some source code"

In [16]:
cache.match_cache_notebook(notebook)

KeyError: 'Cache record not found for NB with hashkey: 07e6a47c8c180cb7851ede6dbb088769'

To understand the difference between an external notebook, and one stored in the cache, we can 'diff' them:

In [17]:
print(cache.diff_nbnode_with_cache(1, notebook, as_str=True))

nbdiff
--- cached pk=1
+++ other: 
## inserted before nb/cells/0:
+  code cell:
+    execution_count: 1
+    source:
+      change some source code
+    outputs:
+      output 0:
+        output_type: stream
+        name: stdout
+        text:
+          1

## deleted nb/cells/0:
-  code cell:
-    execution_count: 1
-    source:
-      a=1
-      print(a)
-    outputs:
-      output 0:
-        output_type: stream
-        name: stdout
-        text:
-          1




If we cache this altered notebook, note that this will not remove the previously cached notebook:

In [18]:
nb_bundle = CacheBundleIn(
    nb=notebook,
    uri=Path("example_nbs", "basic.ipynb"),
    data={"tag": "mytag"}
)
cache.cache_notebook_bundle(nb_bundle)

NbCacheRecord(pk=2)

In [19]:
print(tabulate_cache_records(
    cache.list_cache_records(), path_length=1, hashkeys=True
))

  ID  Origin URI    Created           Accessed          Hashkey
----  ------------  ----------------  ----------------  --------------------------------
   2  basic.ipynb   2022-01-12 15:16  2022-01-12 15:16  07e6a47c8c180cb7851ede6dbb088769
   1  basic.ipynb   2022-01-12 15:15  2022-01-12 15:16  94c17138f782c75df59e989fffa64e3a


Notebooks are retained in the cache, until the cache limit is reached,
at which point the oldest notebooks are removed.

In [20]:
cache.get_cache_limit()

1000

In [21]:
cache.change_cache_limit(100)

(use/api/project)=

## Staging Notebooks for Execution

Notebooks can be staged, by adding the path as a stage record.

```{important}
This does not physically add the notebook to the cache,
merely store its URI, for later use.
```

In [22]:
record = cache.add_nb_to_project(Path("example_nbs", "basic.ipynb"))
record

NbProjectRecord(pk=1)

In [23]:
record.to_dict()

{'uri': '/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb',
 'assets': [],
 'created': datetime.datetime(2022, 1, 12, 15, 16, 27, 64960),
 'traceback': '',
 'read_data': {'name': 'nbformat', 'type': 'plugin'},
 'pk': 1}

If the staged notbook relates to one in the cache, we will be able to retrieve the cache record:

In [24]:
cache.get_cached_project_nb(1)

NbCacheRecord(pk=1)

In [25]:
print(tabulate_project_records(
    cache.list_project_records(), path_length=2, cache=cache
))

  ID  URI                      Reader    Added             Status
----  -----------------------  --------  ----------------  --------
   1  example_nbs/basic.ipynb  nbformat  2022-01-12 15:16  ✅ [1]


We can also retrieve a *merged* notebook.
This is a copy of the source notebook with the following added to it from the cached notebook:

- Selected notebook metadata keys (generally only those keys that affect its execution)
- All code cells, with their outputs and metadata 
  (only selected metadata can be merged if `cell_meta` is not `None`)
  
In this way we create a notebook that is *fully* up-to-date for both its code and textual content:

In [26]:
cache.merge_match_into_file(
    cache.get_project_record(1).uri,
    nb_meta=('kernelspec', 'language_info', 'widgets'),
    cell_meta=None
)

(1,
 {'cells': [{'cell_type': 'markdown',
    'metadata': {},
    'source': '# a title\n\nsome text\n'},
   {'cell_type': 'code',
    'execution_count': 1,
    'metadata': {},
    'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': '1\n'}],
    'source': 'a=1\nprint(a)'}],
  'metadata': {'test_name': 'notebook1',
   'kernelspec': {'display_name': 'Python 3',
    'language': 'python',
    'name': 'python3'},
   'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
    'file_extension': '.py',
    'mimetype': 'text/x-python',
    'name': 'python',
    'nbconvert_exporter': 'python',
    'pygments_lexer': 'ipython3',
    'version': '3.6.1'}},
  'nbformat': 4,
  'nbformat_minor': 2})

If we add a notebook that cannot be found in the cache, it will be listed for execution:

In [27]:
record = cache.add_nb_to_project(Path("example_nbs", "basic_failing.ipynb"))
record

NbProjectRecord(pk=2)

In [28]:
cache.get_cached_project_nb(2)  # returns None

In [29]:
cache.list_unexecuted()

[NbProjectRecord(pk=2)]

In [30]:
print(tabulate_project_records(
    cache.list_project_records(), path_length=2, cache=cache
))

  ID  URI                              Reader    Added             Status
----  -------------------------------  --------  ----------------  --------
   1  example_nbs/basic.ipynb          nbformat  2022-01-12 15:16  ✅ [1]
   2  example_nbs/basic_failing.ipynb  nbformat  2022-01-12 15:17  -


To remove a notebook from the staging area:

In [31]:
cache.remove_nb_from_project(1)

In [32]:
print(tabulate_project_records(
    cache.list_project_records(), path_length=2, cache=cache
))

  ID  URI                              Reader    Added             Status
----  -------------------------------  --------  ----------------  --------
   2  example_nbs/basic_failing.ipynb  nbformat  2022-01-12 15:17  -


(use/api/execute)=

## Execution

If we have some staged notebooks:

In [33]:
cache.clear_cache()
cache.add_nb_to_project(Path("example_nbs", "basic.ipynb"))
cache.add_nb_to_project(Path("example_nbs", "basic_failing.ipynb"))

NbProjectRecord(pk=2)

In [34]:
print(tabulate_project_records(
    cache.list_project_records(), path_length=2, cache=cache
))

  ID  URI                              Reader    Added             Status
----  -------------------------------  --------  ----------------  --------
   1  example_nbs/basic.ipynb          nbformat  2022-01-12 15:17  -
   2  example_nbs/basic_failing.ipynb  nbformat  2022-01-12 15:17  -


Then we can select an executor (specified as entry points) to execute the notebook.

```{tip}
To view the executors log, make sure logging is enabled,
or you can parse a logger directly to `load_executor()`.
```

In [35]:
list_executors()

{'local-parallel', 'local-serial', 'temp-parallel', 'temp-serial'}

In [36]:
from logging import basicConfig, INFO
basicConfig(level=INFO)

executor = load_executor("local-serial", cache=cache)
executor

JupyterExecutorLocalSerial(cache=JupyterCacheBase('/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/.jupyter_cache'))

Calling `run_and_cache()` will run all staged notebooks that do not already have matches in the cache.
It will return a dictionary with lists for:


- **succeeded**: The notebook was executed successfully with no (or only expected) exceptions
- **excepted**: A notebook cell was encountered that raised an unexpected exception
- **errored**: An exception occured before/after the actual notebook execution

```{tip}
Code cells can be tagged with `raises-exception` to let the executor known that
a cell *may* raise an exception (see [this issue on its behaviour](https://github.com/jupyter/nbconvert/issues/730)).
```

```{note}
You can use the `filter_uris` and/or `filter_pks` options to only run selected staged notebooks.
You can also specify the timeout for execution in seconds using the `timeout` option.
```

In [37]:
result = executor.run_and_cache()
result

INFO:jupyter_cache.executors.base:Executing 2 notebook(s) in serial
INFO:jupyter_cache.executors.base:Executing: /Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb
INFO:jupyter_cache.executors.base:Execution Successful: /Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb
INFO:jupyter_cache.executors.base:Executing: /Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic_failing.ipynb
CellExecutionError: An error occurred while executing the following cell:
------------------
raise Exception('oopsie!')
------------------

---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
/var/folders/t2/xbl15_3n4tsb1vr_ccmmtmbr0000gn/T/ipykernel_1308/340246212.py in <module>
----> 1 raise Exception('oopsie!')

Exception: oopsie!
Exception: oopsie!



ExecutorRunResult(succeeded=['/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb'], excepted=['/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic_failing.ipynb'], errored=[])

Successfully executed notebooks will be added to the cache, and data about their execution (such as time taken) will be stored in the cache record:

In [38]:
cache.list_cache_records()

[NbCacheRecord(pk=1)]

In [39]:
record = cache.get_cache_record(1)
record.to_dict()

{'description': '',
 'hashkey': '94c17138f782c75df59e989fffa64e3a',
 'created': datetime.datetime(2022, 1, 12, 15, 17, 45, 471862),
 'accessed': datetime.datetime(2022, 1, 12, 15, 17, 45, 471871),
 'data': {'execution_seconds': 1.8344826350000005},
 'uri': '/Users/chrisjsewell/Documents/GitHub/jupyter-cache/docs/using/example_nbs/basic.ipynb',
 'pk': 1}

Notebooks which failed to run will **not** be added to the cache,
but details about their execution (including the exception traceback)
will be added to the stage record:

In [40]:
record = cache.get_project_record(2)
print(record.traceback)

Traceback (most recent call last):
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/jupyter_cache/executors/utils.py", line 58, in single_nb_execution
    executenb(
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/.tox/py38/lib/python3.8/site-packages/nbclient/client.py", line 1093, in execute
    return NotebookClient(nb=nb, resources=resources, km=km, **kwargs).execute()
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/.tox/py38/lib/python3.8/site-packages/nbclient/util.py", line 84, in wrapped
    return just_run(coro(*args, **kwargs))
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/.tox/py38/lib/python3.8/site-packages/nbclient/util.py", line 62, in just_run
    return loop.run_until_complete(coro)
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/.tox/py38/lib/python3.8/site-packages/nest_asyncio.py", line 81, in run_until_complete
    return f.result()
  File "/Users/chrisjsewell/Documents/GitHub/jupyter-cache/.tox/py38/lib/pytho

We now have two staged records, and one cache record:

In [41]:
print(tabulate_project_records(
    cache.list_project_records(), path_length=2, cache=cache
))

  ID  URI                              Reader    Added             Status
----  -------------------------------  --------  ----------------  --------
   1  example_nbs/basic.ipynb          nbformat  2022-01-12 15:17  ✅ [1]
   2  example_nbs/basic_failing.ipynb  nbformat  2022-01-12 15:17  ❌


In [42]:
print(tabulate_cache_records(
    cache.list_cache_records(), path_length=1, hashkeys=True
))

  ID  Origin URI    Created           Accessed          Hashkey
----  ------------  ----------------  ----------------  --------------------------------
   1  basic.ipynb   2022-01-12 15:17  2022-01-12 15:17  94c17138f782c75df59e989fffa64e3a


### Timeout
A **timeout** argument can also be passed to `run_and_cache()` which takes value in seconds.
Alternatively, timeout can also be specified inside the notebook metadata:

```
'execution': {
   'timeout': 30
 }
```
```{note}
Timeout specified in notebook metadata will take precedence over the one passed as an argument to `run_and_cache()`.
```